In [1]:
import sys, os
from pathlib import Path
import ee
from joblib import Memory
from google.cloud import storage

sys.path.append(os.path.abspath(".."))
# Resolve all data paths relative to the repo root
REPO_ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parents[0]

from config import load_config
from gee_utils import (
    init_gee,
    load_image_collection,
    # Vector workflow functions
    upload_parcels_to_gcs,
    ingest_parcels_to_gee,
    reduce_regions_to_gcs,
    # Raster workflow functions (for reference)
    upload_raster_to_gcs,
    ingest_raster_to_gee,
)
from data_utils import upload_to_gcs
from mappluto import load_and_sample
from geometry import get_city_geometry
from naip import calculate_spectral_indices
from raster_utils import rasterize_parcels
from analysis import (
    read_csv_from_gcs,
    load_spectral_stats,
    cluster_dataframe,
    analyze_cluster_composition,
)

In [2]:
# ===== RUN CONFIGURATION =====
CONFIG_FILE = "nyc_vacant.yaml"  # Change to "nyc_buildings", etc.
# =============================

memory = Memory(Path("cache"), verbose=0)

# Load run-specific config
CONFIG = load_config(CONFIG_FILE, config_dir=REPO_ROOT / "config")

# Create output directories
CONFIG.ensure_output_dirs(REPO_ROOT)

# gcs client
client = storage.Client()
bucket = client.bucket(CONFIG.gcp.bucket)

23:31:54 | INFO | vacant_lots | Loaded config for: nyc_vacant.yaml


In [3]:
init_gee(CONFIG.gcp)

23:31:56 | INFO | vacant_lots | Initializing GEE with ADC credentials and project:vacant-lot-detection
23:31:57 | INFO | vacant_lots | GEE sucessfully initialized


In [4]:
"""
- Load MapPLUTO (NYC) 2022 v3 -- fall matches with NAIP
- Reproject to output_crs (EPSG:32618) for geometry-derived metric area computation
- Filter by area: min from min_pixels × resolution²
- Sample vacant lots based on land use codes (configurable)
- Convert CRS: EPSG:32618 --> EPSG:4326 for GEE
"""
mappluto_22_gdf, sampled_gdf = load_and_sample(
    path=CONFIG.get_parcel_path(REPO_ROOT),
    layer=CONFIG.parcel.layer,
    land_use_codes=CONFIG.parcel.vacant_codes,
    col_to_sample=CONFIG.parcel.landuse_column,
    projected_crs=CONFIG.raster.output_crs,
    vacant_min_fraction=CONFIG.sampling.vacant_min_fraction,
    total_samples=CONFIG.sampling.total_samples,
    resolution=CONFIG.raster.resolution,
    min_pixels=CONFIG.sampling.min_pixels,
    random_state=CONFIG.sampling.random_state,
)

23:31:59 | INFO | vacant_lots | Loaded /Users/joyadebi/repos/Vacant_Lot_Detection/data/nyc_mappluto_22v3_arc_fgdb/MapPLUTO22v3.gdb
23:32:03 | INFO | vacant_lots | Computed 'area_m2_epsg32618' and 'geom_perimeter_epsg32618' in EPSG:32618
23:32:04 | INFO | vacant_lots | min_area_m2: 50.0 m² (50 pixels × 1.0² m) — 852,800 parcels after min filter
23:32:05 | INFO | vacant_lots | Sampled 25,000 parcels (2000 target ['11'], 23000 non-target)
23:32:05 | INFO | vacant_lots | Converting CRS: EPSG:32618 --> EPSG:4326


In [5]:
city_geom = get_city_geometry(CONFIG)

@memory.cache
def get_naip_for_city(config):
    return load_image_collection(
        collection_id=config.naip.collection_id,
        start_date=config.naip.start_date,
        end_date=config.naip.end_date,
        region=city_geom,
        mosaic=True
    )

naip = get_naip_for_city(CONFIG)
naip = calculate_spectral_indices(naip, CONFIG)

23:32:10 | INFO | vacant_lots | scaling ['R', 'G', 'B', 'N'] by 255.0
23:32:10 | INFO | vacant_lots | calculating NDVI
23:32:10 | INFO | vacant_lots | scaling NDVI to [0,1]
23:32:10 | INFO | vacant_lots | calculating SAVI
23:32:10 | INFO | vacant_lots | clamping SAVI values [-1, 1]
23:32:10 | INFO | vacant_lots | scaling SAVI values to [0,1]
23:32:10 | INFO | vacant_lots | calculating brightness
23:32:10 | INFO | vacant_lots | Added NDVI, SAVI, BRIGHTNESS bands
23:32:10 | INFO | vacant_lots | Added Bare Soil Proxy bands


# Vector Based Reduction

## Parcel Table Ingestion to GEE
- run when parcels should be overwritten in both gcs and gee 

In [ ]:
# gcs_uri = upload_parcels_to_gcs(
#     gdf=sampled_gdf,
#     id_column=CONFIG.parcel.id_column,
#     output_dir=CONFIG.get_intermediaries_dir(REPO_ROOT),
#     bucket_name=CONFIG.gcp.bucket,
#     gcs_prefix=CONFIG.gee.export_prefix,
#     filename_prefix=f"parcels",
# )

23:33:29 | INFO | vacant_lots | Created minimal GDF with 25000 parcels, ID column 'BBL' as string
23:33:29 | INFO | vacant_lots | Wrote shapefile: /Users/joyadebi/repos/Vacant_Lot_Detection/outputs/eda/nyc_vacant/intermediaries/parcels.shp
23:33:30 | INFO | vacant_lots | Created zip: /Users/joyadebi/repos/Vacant_Lot_Detection/outputs/eda/nyc_vacant/intermediaries/parcels.zip
23:33:30 | INFO | vacant_lots | Uploading /Users/joyadebi/repos/Vacant_Lot_Detection/outputs/eda/nyc_vacant/intermediaries/parcels.zip to gs://thesis_parcels/eda/new_york_new_york/parcels.zip
23:33:32 | INFO | vacant_lots | Upload complete: gs://thesis_parcels/eda/new_york_new_york/parcels.zip
Uploaded: gs://thesis_parcels/eda/new_york_new_york/parcels.zip


In [ ]:
"""
Can provide bucket_name, gcs_prefix, gcs_filename if cell above doesn't need to be re run 
f"gs://{bucket_name}/{gcs_prefix}/{gcs_filename}"
"""
# ingest_parcels_to_gee(
#     asset_id=f"projects/{CONFIG.gcp.project_id}/assets/{CONFIG.city}_parcels",
#     gcs_uri=gcs_uri,
# )

'\nCan provide bucket_name, gcs_prefix, gcs_filename if cell above doesn\'t need to be re run \nf"gs://{bucket_name}/{gcs_prefix}/{gcs_filename}"\n'

# Rasterize Parcels

In [ ]:
# raster_output_path = CONFIG.get_raster_path(REPO_ROOT)
# intermediaries_dir = CONFIG.get_intermediaries_dir(REPO_ROOT)
# mapping_output_path = intermediaries_dir / "parcel_id_mapping.csv"

# raster_path, mapping_path = rasterize_parcels(
#     gdf=mappluto_22_gdf,
#     id_column=CONFIG.parcel.id_column,
#     raster_output_path=raster_output_path,
#     mapping_output_path=mapping_output_path,
#     crs=CONFIG.raster.output_crs,
#     resolution=CONFIG.raster.resolution,
#     overwrite=False,
# )

The following cell should only be run once when the raster needs to be uploaded to gcs and then ingested to gee

In [ ]:
# ## Upload parcel raster to GEE

# # Upload to GCS first
# gcs_uri = upload_raster_to_gcs(
#     raster_path=raster_path,
#     gcs_bucket=CONFIG.gcp.bucket,
#     gcs_prefix=CONFIG.gee.export_prefix,
#     gcs_filename="parcels_raster.tif",
# )

# # Ingest to GEE asset
# task_info = ingest_raster_to_gee(
#     gcs_uri=gcs_uri,
#     asset_id=CONFIG.gee.parcel_asset,
# )

# # Task runs asynchronously - check progress in Earth Engine Code Editor
# print(f"Task {task_info['task_id']} submitted")
# print(f"Check progress at: https://code.earthengine.google.com/tasks")

## Raster-Based Parcel Reduction

Instead of using `reduceRegions()` with vector polygons (which is slow for many parcels), we can use a grouped reducer with the parcel raster. This approach:
1. Uploads the parcel raster to GEE as an Image asset
2. Stacks the imagery bands with the parcel ID band
3. Uses `reduceRegion()` with a grouped reducer to compute stats per parcel ID

Only upload to GEE when raster asset for a specific city needs to be overwritten. This is a time consuming process.

In [ ]:
# # Raster-based grouped reducer with ONLY sampled parcels
# from gee_utils import load_parcel_raster_asset, reduce_by_parcel_raster, export_grouped_stats_to_gcs
# import pandas as pd

# # Load parcel raster from GEE
# parcel_raster_asset = CONFIG.gee.parcel_asset
# parcel_raster = load_parcel_raster_asset(parcel_raster_asset)

# # Load the parcel_id mapping (BBL -> rasterized integer ID)
# mapping_path = CONFIG.get_intermediaries_dir(REPO_ROOT) / "parcel_id_mapping.csv"
# parcel_mapping = pd.read_csv(mapping_path)
# print(f"Loaded parcel mapping: {len(parcel_mapping)} parcels")

# # Get the rasterized IDs for sampled parcels only
# # Join sampled_gdf with mapping on BBL
# sampled_bbls = sampled_gdf['BBL'].astype(int).tolist()
# sampled_mapping = parcel_mapping[parcel_mapping['BBL'].isin(sampled_bbls)]
# sampled_parcel_ids = sampled_mapping['parcel_id'].tolist()
# print(f"Sampled parcels with mapping: {len(sampled_parcel_ids)}")

# # Verify bands
# print("NAIP bands:", naip.bandNames().getInfo())
# bands_to_reduce = ['R', 'G', 'B', 'N', 'NDVI', 'SAVI', 'Brightness', 'BareSoilProxy']

# naip_32618 = naip.reproject("EPSG:32618")
# # Run reduction with ONLY sampled parcel IDs
# grouped_stats = reduce_by_parcel_raster(
#     imagery=naip_32618,
#     parcel_raster=parcel_raster,
#     region=city_geom,
#     bands=bands_to_reduce,
#     scale=1,
#     debug=True,
#     parcel_ids=sampled_parcel_ids,  # Only process sampled parcels!
# )

# # Export results to GCS with new path structure
# task = export_grouped_stats_to_gcs(
#     stats_dict=grouped_stats,
#     description='parcel_spectral_stats_raster',
#     bucket=CONFIG.gcp.bucket,
#     file_prefix=f'{CONFIG.gee.export_prefix}/parcel_spectral_stats_raster',  # New path: eda/new_york_new_york/
# )

# Previous Vector based approach to reducing

In [ ]:
ingest_parcels_to_gee(
    asset_id=f"projects/{CONFIG.gcp.project_id}/assets/{CONFIG.city}_parcels",
    bucket_name=CONFIG.gcp.bucket,
    gcs_prefix=CONFIG.gee.export_prefix,
    gcs_filename=f"{CONFIG.city}_parcels.zip",
)

In [ ]:
naip.bandNames().getInfo()

In [ ]:
# load parcels from GEE (vector approach - kept for comparison)
parcels = ee.FeatureCollection(f"projects/{CONFIG.gcp.project_id}/assets/nyc_parcels")

reducer = (
    ee.Reducer.mean()
      .combine(ee.Reducer.median(), "", True)
      .combine(ee.Reducer.stdDev(), "", True)
      .combine(ee.Reducer.count(), "", True)
)

stats = naip.reduceRegions(
    collection=parcels,
    reducer=reducer,
    scale=1,            # NAIP is 1 m
    tileScale=4         # optional: helps avoid memory errors
)

task = ee.batch.Export.table.toCloudStorage(
    collection=stats,
    description='parcel_spectral_stats',
    bucket=CONFIG.gcp.bucket,
    fileNamePrefix=f'{CONFIG.gee.export_prefix}/parcel_spectral_stats',
    fileFormat='CSV'
)
task.start()

# NOTE: Consider using the raster-based approach below for better performance

## Start here to get the Parcel by Parcel Spectral Stats

In [ ]:
from io import BytesIO
import pandas as pd
#read csv back from gcs
bucket = client.bucket(CONFIG.gcp.bucket)
blob = bucket.blob(f"{CONFIG.gee.export_prefix}/parcel_spectral_stats.csv")
spectral_stats = blob.download_as_bytes()
spectral_df = pd.read_csv(BytesIO(spectral_stats))


spectral_df

In [ ]:
mappluto_features = sampled_gdf[["BBL", "LandUse", "geometry", "geom_perimeter_epsg32618", "Shape_Area"]].copy()
mappluto_features

In [ ]:
# spectral only clustering 
mappluto_spectral_df = mappluto_features[["BBL", "LandUse"]].copy()

#join on bbl with spectral df 
merged_df = spectral_df.merge(mappluto_spectral_df, on="BBL", how="left")
merged_df

In [ ]:
# Preview the spectral features that will be used for clustering
# (The actual feature extraction is handled by the analysis module)
feature_cols = CONFIG.clustering.features if CONFIG.clustering else [c for c in merged_df.columns if c.endswith("_mean")]
print(f"Features for clustering: {feature_cols}")
merged_df[feature_cols].describe()

In [ ]:
# Using the analysis module for clustering
# This replaces the manual KMeans + PCA code with the formalized pipeline

merged_df = cluster_spectral_data(
    df=merged_df,
    feature_columns=CONFIG.clustering.features if CONFIG.clustering else None,
    n_clusters=CONFIG.clustering.n_clusters if CONFIG.clustering else 5,
    random_state=CONFIG.clustering.random_state if CONFIG.clustering else 42,
    add_pca=True,
)
merged_df

In [ ]:
import matplotlib.pyplot as plt

# unique categories
landuse_classes = sorted(merged_df["LandUse"].unique())
clusters = sorted(merged_df["cluster"].unique())

# assign each landuse a color
cmap = plt.get_cmap("tab20")
color_map = {lu: cmap(i % 20) for i, lu in enumerate(landuse_classes)}

# assign each cluster a marker
markers = ["o", "s", "^", "v", "P", "D", "*", "X", "<", ">"]
marker_map = {cl: markers[i % len(markers)] for i, cl in enumerate(clusters)}

plt.figure(figsize=(10, 8))

for cl in clusters:
    subset = merged_df[merged_df["cluster"] == cl]
    plt.scatter(
        subset["pc1"],
        subset["pc2"],
        c=subset["LandUse"].map(color_map),
        marker=marker_map[cl],
        s=6,
        alpha=0.75,
        label=f"Cluster {cl}"
    )

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PC Space: Color = Landuse, Marker = Cluster")

# legend for cluster markers
plt.legend(title="Cluster", bbox_to_anchor=(1.05, 1), loc="upper left")

plt.show()


In [ ]:
# Analyze cluster composition using the analysis module
composition = analyze_cluster_composition(merged_df, cluster_col="cluster", category_col="LandUse")

composition.plot(kind="bar", stacked=True, figsize=(10,6), colormap="tab20")
plt.title("Landuse composition per cluster")
plt.ylabel("Fraction")
plt.show()

# Show the composition table
composition

In [ ]:
#vegetation fraction 
# Optional but safe:

# number of NAIP pixels in parcel

# vegetation fraction (proportion NDVI > 0.2 or SAVI > 0.25)

In [ ]:
# stats by explicit landuse
mappluto_22_gdb[mappluto_22_gdb["LandUse"] == "11"]["Shape_Area"].describe()
mean = mappluto_22_gdb[mappluto_22_gdb["LandUse"] == "11"]["Shape_Area"].mean()
std = mappluto_22_gdb[mappluto_22_gdb["LandUse"] == "11"]["Shape_Area"].std()

lower_bound = mean - std
upper_bound = mean + std

within_1std = mappluto_22_gdb[
    (mappluto_22_gdb["Shape_Area"] >= lower_bound) &
    (mappluto_22_gdb["Shape_Area"] <= upper_bound)
]
areas = within_1std["Shape_Area"]
within_1std["Shape_Area"].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
plt.hist(areas, bins=100, color="skyblue", edgecolor="black", alpha=0.7)
plt.yscale("log")  # log scale helps reveal distribution
plt.xlabel("Parcel Area (sqft)")
plt.ylabel("Count (log scale)")
plt.title("Distribution of Vacant Lot Areas Within 1 Std dev (MapPLUTO)")

mean = areas.mean()
std = areas.std()
median = areas.median()
plt.axvline(mean, color="red", linestyle="--", label=f"Mean = {mean:.0f}")
plt.axvline(mean + std, color="orange", linestyle="--", label="+1 Std")
plt.axvline(mean - std, color="orange", linestyle="--", label="-1 Std")
plt.axvline(median, color="blue", linestyle="--", label="median")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# stats by NumBldgs, BldgArea, BuiltFar
# Due to LandUse counts
potential_vacant, lots_to_inspect, vacant_landuse_counts = identify_potential_vacant_lots(mappluto_22_gdb)
potential_vacant["Shape_Area"].describe()

In [ ]:
# Makes sense
# 7: Transportation and Utility
# 9: Open space and Outdoor Recreation
# 10: Parking Facilities
# Doesn't Really Make Sense
# 01,02,03,04,05: Buildings
# 08: Public Facilities and Institutions
# TODO kind of want to look at these lots in pictures
vacant_landuse_counts

In [ ]:
# Generate README for this run
from config import generate_run_readme

run_stats = {
    "Total parcels loaded": len(mappluto_22_gdf),
    "Sampled parcels": len(sampled_gdf),
    "Land use codes": sorted(sampled_gdf['LandUse'].unique()),
    "Unique clusters": len(merged_df['cluster'].unique()),
    "Total spectral features": len(CONFIG.clustering.features),
}

generate_run_readme(
    config=CONFIG,
    output_dir=CONFIG.get_output_dir(REPO_ROOT),
    stats=run_stats,
)